In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__)

!mkdir /root/.kaggle/
with open("/root/.kaggle/kaggle.json", "w") as file:
    file.write('{"username":"burindima","key":"3dc92a9952274c1e89f2f8a09d84f9c0"}')
!kaggle competitions download -c ml-intensive-of-yandex-academy-2023
!unzip ml-intensive-of-yandex-academy-2023.zip 

train_data = pd.read_pickle('data_train')
test_data = pd.read_pickle('data_test')
x_test = test_data['images'] / 255.0

print(f'Train data keys: {train_data.keys()}')
print(f'Test data keys: {test_data.keys()}\n')
print(f'Shape of train images: {train_data["images"].shape}')
print(f'Shape of test images: {test_data["images"].shape}\n')
print(f'Unique labels: {set(train_data["labels"])}\n')
print(f'Classes distribution: {pd.Series(train_data["labels"]).value_counts(normalize=True)}')

In [ ]:
orig_ds = tf.data.Dataset.from_tensor_slices((train_data['images'], train_data['labels']))
test_ds = tf.data.Dataset.from_tensor_slices((test_data['images']))

In [ ]:
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE
IMG_SIZE = 32

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

def prepare(ds, shuffle=False, augment=False):
  # Resize and rescale all datasets.
  ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets.
  ds = ds.batch(batch_size).map(lambda x, y: (x, tf.one_hot(y, depth=10)))

  # Use buffered prefetching on all datasets.
  return ds.prefetch(buffer_size=AUTOTUNE)

def prepare_test(ds):
  return ds.map(lambda x: resize_and_rescale(x), 
              num_parallel_calls=AUTOTUNE)

In [ ]:
orig_ds = augmented_train_dataset
len_orig_ds = len(orig_ds)
orig_val_ds, orig_train_ds = orig_ds.take(int(len_orig_ds* 0.15)) , orig_ds.skip(int(len_orig_ds * 0.15))

In [ ]:
full_train_ds = prepare(orig_train_ds, shuffle=True)
val_ds = prepare(orig_val_ds)
test_ds = prepare_test(test_ds)

In [ ]:
!pip install livelossplot

In [ ]:
from tensorflow.keras import layers

#СОЗДАЕМ МОДЕЛЬ 
input_shape = (32, 26, 3)

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, BatchNormalization, Activation, Add,Dropout


# Определение функции для создания Residual блоков
def residual_block(inputs, num_filters, kernel_size, strides):
    x = Conv2D(num_filters, kernel_size=kernel_size, strides=strides, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Add()([x, inputs])
    x = Activation('relu')(x)
    return x

# Создание модели ResNet-50 архитектуры
inputs = Input(shape=input_shape)
x = Conv2D(64, kernel_size=3, strides=1, padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = residual_block(x, 64, 3, 1)
x = residual_block(x, 64, 3, 1)
x = residual_block(x, 64, 3, 1)
x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Activation('relu')(x)
x = residual_block(x, 128, 3, 1)
x = residual_block(x, 128, 3, 1)
x = residual_block(x, 128, 3, 1)
x = Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Activation('relu')(x)
x = residual_block(x, 256, 3, 1)
x = residual_block(x, 256, 3, 1)
x = residual_block(x, 256, 3, 1)
x = Flatten()(x)
outputs = Dense(10, activation='softmax')(x)
model = Model(inputs, outputs)


# Компиляция и обучение модели
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
initial_learning_rate = 0.01
epochs = 100
decay = initial_learning_rate / epochs
def lr_time_based_decay(epoch, lr):
 return lr * 1 / (1 + decay * epoch)


In [ ]:
from livelossplot import PlotLossesKeras
import time 

start = time.time()
callback_ = tf.keras.callbacks.LearningRateScheduler(lr_time_based_decay, verbose=1) 
model_history = model.fit(full_train_ds, validation_data=val_ds, epochs=100, callbacks=[PlotLossesKeras(), callback], verbose=False)
end = time.time() - start 
print("Время: "+ str(end) +' секунд')